# Problem Statement

The data given is of the mutual funds in the USA. The objective of this problem is to predict the ‘basis point spread’ over AAA bonds i.e. feature ‘bonds_aaa’ against each Serial Number.
Basis Point Spread indicates the additional return a mutual fund would give over the AAA-rated bonds.

## About the Dataset

For this task, we have only taken the required columns and dropped the unnecessary columns. The data has already been cleansed for better analysis.

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns
import matplotlib.pyplot as plt
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV,train_test_split
from sklearn.metrics import mean_squared_error

In [25]:
dict=pd.read_excel('MutualFundReturnsDataDictionary.xlsx')
dict

,Features,Description
0,Fund Symbol,Uniques symbol for the mutual fund used for re...
1,Fund Name,Full name of the mutual fund scheme
2,Category,Investment category of the mutual fund
3,Fund Family,Asset management company to which mutual fund ...
4,Investment,Type of investment of the mutual fund scheme
5,Size,Size of the mutual fund based on the total net...
6,Total net assets,Total assets under management for the mutual f...
7,Currency,Currency in which the investments of the mutua...
8,Net Annual Expense Ratio,Expense ratio is the fee that the asset manage...
9,Morningstar Rating,This is the overall fund rating given by the r...


In [4]:
data = pd.read_csv('train.csv')
data.drop(columns=['Id', 'Serial Number'], inplace=True)

In [5]:
data.head()

,total_net_assets,net_annual_expenses_ratio,morningstar_rating,portfolio_cash,portfolio_stocks,portfolio_bonds,portfolio_others,portfolio_preferred,portfolio_convertable,sectors_basic_materials,...,category_102,category_103,category_104,category_105,category_106,category_107,investment_0,investment_1,investment_2,bonds_aaa
0,211000,1.90,3,0.95,99.05,0.00,0.00,0.00,0.00,3.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.00
1,98930000,1.57,2,0.00,0.00,100.00,0.00,0.00,0.00,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.65
2,440000000,0.53,4,1.71,0.00,98.29,0.00,0.00,0.00,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,93.20
3,56050000,0.93,3,2.64,31.29,57.06,0.80,7.62,0.58,3.07,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2.64
4,173040000,2.26,2,5.24,94.75,0.00,0.01,0.00,0.00,9.42,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.00


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9518 entries, 0 to 9517
Columns: 152 entries, total_net_assets to bonds_aaa
dtypes: float64(148), int64(4)
memory usage: 11.0 MB


In [7]:
data.columns

Index(['total_net_assets', 'net_annual_expenses_ratio', 'morningstar_rating',
       'portfolio_cash', 'portfolio_stocks', 'portfolio_bonds',
       'portfolio_others', 'portfolio_preferred', 'portfolio_convertable',
       'sectors_basic_materials',
       ...
       'category_102', 'category_103', 'category_104', 'category_105',
       'category_106', 'category_107', 'investment_0', 'investment_1',
       'investment_2', 'bonds_aaa'],
      dtype='object', length=152)

In [8]:
X = data.iloc[:,:-1]
y = data.iloc[:,-1]

In [9]:
scaler = MinMaxScaler()
data_scaled = pd.DataFrame(scaler.fit_transform(X))
data_scaled.columns = X.columns

In [10]:
hyper_parameter = {
    'n_estimators ':[100,150],
    'max_depth':[1,2,3],
    'learning_rate':[0.1,1]
}

In [11]:
xgb = XGBRegressor(random_state=0)
xgb_grid = GridSearchCV(estimator=xgb, param_grid=hyper_parameter, scoring='neg_root_mean_squared_error')

In [12]:
xgb_grid.fit(X, y)

[17:59:12] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { n_estimators  } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[17:59:13] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { n_estimators  } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[17:59:14] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { n_estimators  } might not be used.

  This may not be accurate due to some parameters are only used in

[17:59:48] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { n_estimators  } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[17:59:51] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { n_estimators  } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[17:59:53] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { n_estimators  } might not be used.

  This may not be accurate due to some parameters are only used in

[18:00:29] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { n_estimators  } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[18:00:31] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { n_estimators  } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[18:00:33] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { n_estimators  } might not be used.

  This may not be accurate due to some parameters are only used in

GridSearchCV(estimator=XGBRegressor(base_score=None, booster=None,
                                    colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, gamma=None,
                                    gpu_id=None, importance_type='gain',
                                    interaction_constraints=None,
                                    learning_rate=None, max_delta_step=None,
                                    max_depth=None, min_child_weight=None,
                                    missing=nan, monotone_constraints=None,
                                    n_estimators=100, n_jobs=None,
                                    num_parallel_tree=None, random_state=0,
                                    reg_alpha=None, reg_lambda=None,
                                    scale_pos_weight=None, subsample=None,
                                    tree_method=None, validate_parameters=None,
         

In [13]:
test = pd.read_csv('test.csv')
Id = pd.DataFrame(test.Id)
test.drop(columns=['Id','Serial Number'], inplace=True)

In [14]:
test_scaled = pd.DataFrame(scaler.transform(test))

In [15]:
test_scaled

,0,1,2,3,4,5,6,7,8,9,...,141,142,143,144,145,146,147,148,149,150
0,0.019202,0.035361,0.6,0.0004,0.0000,0.9898,0.010347,0.00000,0.000000,0.0000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,0.005372,0.062235,0.6,0.2492,0.3056,0.4406,0.000950,0.00406,0.000853,0.0465,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.001719,0.158416,0.2,0.3470,0.6281,0.0000,0.026291,0.00000,0.000000,0.0437,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.000028,0.240453,0.4,0.0961,0.0000,0.9040,0.000000,0.00000,0.000000,0.0000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.000025,0.029703,0.0,0.0029,0.9971,0.0000,0.000000,0.00000,0.000000,0.0290,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2375,0.000917,0.000000,0.0,0.0118,0.0637,0.0140,0.961145,0.00014,0.000107,0.0269,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2376,0.000553,0.056577,0.8,0.0045,0.0000,0.9955,0.000000,0.00000,0.000000,0.0000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2377,0.000168,0.144272,0.8,0.0907,0.6680,0.2413,0.000000,0.00000,0.000000,0.0588,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2378,0.049840,0.270156,1.0,0.0221,0.9689,0.0000,0.009503,0.00000,0.000000,0.0082,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0


In [16]:
predictions = xgb_grid.predict(test)

In [17]:
Id['bonds_aaa'] = predictions

In [19]:
Id.to_csv('submission.csv', index=False)